In [1]:
import pandas as pd
import numpy as np

In [3]:
data = pd.ExcelFile('../data/datos.xlsx')

In [4]:

df = data.parse('Consumo')  

df_cleaned = df.dropna(how='all')

tablas_config = {
    'Cocaína': {
        'start_col': 0,
        'cols': ['País', 'Cocaína', 'Categoría_Cocaína']
    },
    'Cannabis': {
        'start_col': 4,
        'cols': ['País', 'Cannabis', 'Categoría_Cannabis']
    },
    'Metanfetaminas': {
        'start_col': 8,
        'cols': ['País', 'Metanfetaminas', 'Categoría_Metanfetaminas']
    },
    'Opioides': {
        'start_col': 12,
        'cols': ['País', 'Opioides', 'Categoría_Opioides']
    }
}

def procesar_tabla(df, start_col, nombres_columnas):
    """Procesa cada tabla individual respetando tu estructura real"""
    try:
        tabla = df.iloc[2:, start_col:start_col+3].copy()
        tabla.columns = nombres_columnas
        
        tabla = tabla.dropna(subset=['País'])
        col_valor = nombres_columnas[1]
        tabla[col_valor] = pd.to_numeric(tabla[col_valor], errors='coerce')
        
        return tabla[['País', col_valor]]
    
    except Exception as e:
        print(f"Error procesando {nombres_columnas[1]}: {str(e)}")
        return pd.DataFrame()

dataframes = {}
for droga, config in tablas_config.items():
    temp_df = procesar_tabla(df_cleaned, 
                           config['start_col'], 
                           config['cols'])
    dataframes[droga] = temp_df

df_final = dataframes['Cocaína']
for droga in ['Cannabis', 'Metanfetaminas', 'Opioides']:
    df_final = df_final.merge(dataframes[droga], 
                            on='País', 
                            how='outer')

def aplicar_categorias(df):
    categorizadores = {
        'Cocaína': [
            (1.2, 'Alta'),
            (0.6, 'Media'),
            (0.29, 'Baja'),
            (-np.inf, 'Muy baja')
        ],
        'Cannabis': [
            (6.8, 'Alta'),
            (3.9, 'Media'),
            (2.3, 'Baja'),
            (-np.inf, 'Muy baja')
        ],
        'Metanfetaminas': [
            (0.9, 'Alta'),
            (0.48, 'Media'),
            (0.17, 'Baja'),
            (-np.inf, 'Muy Baja')
        ],
        'Opioides': [
            (1, 'Alta'),
            (0.5, 'Media'),
            (0.26, 'Baja'),
            (-np.inf, 'Muy baja')
        ]
    }
    
    for droga, limites in categorizadores.items():
        col_categoria = f'Categoría_{droga}'
        df[col_categoria] = df[droga].apply(
            lambda x: next((cat for lim, cat in limites if x > lim), 'Sin dato')
        )
    
    return df

df_final = aplicar_categorias(df_final)

column_order = ['País'] + \
    [f'{d}|Categoría_{d}' for d in tablas_config.keys()]
    
df_final = df_final.fillna(0).sort_values('País').reset_index(drop=True)

In [5]:
df_final.to_csv('../data/consumo.csv', index=False)

In [ ]:
df_produccion = data.parse('Producción', header=None)

df_prod_cleaned = df_produccion.dropna(how='all')

def procesar_tabla_compleja(df, start_row, start_col, columnas):
    tabla = df.iloc[start_row:, start_col:start_col+3].copy()
    tabla.columns = columnas
    tabla = tabla.dropna(subset=[columnas[0]])
    tabla[columnas[1]] = pd.to_numeric(tabla[columnas[1]], errors='coerce')
    return tabla

cultivo_config = {
    'start_row': 4,  
    'start_col': 0,
    'columnas': ['País', 'Coca', 'Opio']
}

df_cultivo = procesar_tabla_compleja(
    df_prod_cleaned,
    cultivo_config['start_row'],
    cultivo_config['start_col'],
    cultivo_config['columnas']
)

df_cultivo = df_cultivo[df_cultivo['País'].str.strip().ne('')]
df_cultivo[['Coca', 'Opio']] = df_cultivo[['Coca', 'Opio']].apply(lambda x: pd.to_numeric(x.astype(str).str.replace('.', '').str.replace(',', '.'), errors='coerce'))

manufactura_config = {
    'laboratorios': [
        {
            'nombre': 'Cocaína',
            'start_row': 2,
            'start_col': 2,
            'columnas': ['País', 'Laboratorios_Cocaina', 'Categoría_Cocaina']
        },
        {
            'nombre': 'Opioides',
            'start_row': 2,
            'start_col': 6,
            'columnas': ['País', 'Laboratorios_Opioides', 'Categoría_Opioides']
        },
        {
            'nombre': 'Metanfetaminas',
            'start_row': 2,
            'start_col': 10,
            'columnas': ['País', 'Laboratorios_Metanfetaminas', 'Categoría_Metanfetaminas']
        }
    ]
}

dfs_manufactura = []
for config in manufactura_config['laboratorios']:
    df_temp = procesar_tabla_compleja(
        df_prod_cleaned,
        config['start_row'],
        config['start_col'],
        config['columnas']
    )
    
    df_temp = df_temp[~df_temp.iloc[:, 1].astype(str).str.contains('NaN|^\s*$')]
    df_temp['País'] = df_temp['País'].str.strip()
    
    dfs_manufactura.append(df_temp)

df_manufactura = dfs_manufactura[0]
for df in dfs_manufactura[1:]:
    df_manufactura = df_manufactura.merge(df, on='País', how='outer')

categorias_map = {
    'ALTA': 'Alta',
    'MEDIA': 'Media',
    'BAJA': 'Baja',
    'MUY BAJA': 'Muy baja',
    'MUY BAJO': 'Muy baja'
}

for col in df_manufactura.filter(like='Categoría').columns:
    df_manufactura[col] = df_manufactura[col].str.strip().replace(categorias_map).fillna('Sin dato')



/tmp/ipykernel_3174/3694228754.py:26: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_cultivo[['Coca', 'Opio']] = df_cultivo[['Coca', 'Opio']].apply(lambda x: pd.to_numeric(x.astype(str).str.replace('.', '').str.replace(',', '.'), errors='coerce'))


In [7]:
df_cultivo.to_csv('../data/cultivo.csv', index=False)

In [8]:
df_manufactura.to_csv('../data/produccion.csv', index=False)

In [9]:
df_trafico = data.parse('Tráfico', header=None)

df_trafico_clean = df_trafico.dropna(how='all')

header_row = 1  # La fila REAL con los encabezados es la 1 (índice 1)
df_trafico_clean.columns = ['Col_extra', 'País', 'Cocaína', 'Cannabis', 'Metanfetaminas', 'Heroína']

df_trafico_clean = df_trafico_clean.drop(columns=['Col_extra'])

df_trafico_final = df_trafico_clean.iloc[2:].copy()

df_trafico_final = df_trafico_final.reset_index(drop=True)
df_trafico_final['País'] = df_trafico_final['País'].str.strip()

def limpiar_valores(valor):
    if isinstance(valor, str):
        try:
            return float(valor.replace('$', '').replace(',', '').strip())
        except:
            return np.nan
    return valor

for col in ['Cocaína', 'Cannabis', 'Metanfetaminas', 'Heroína']:
    df_trafico_final[col] = df_trafico_final[col].apply(limpiar_valores)

In [10]:
df_trafico_final.to_csv('../data/trafico.csv', index=False)

In [11]:
df_mercado = data.parse('Estimación de Mercado', header=None)
df_mercado_clean = df_mercado.dropna(how='all', axis=0)


def procesar_columna_ajustada(df, start_col):
    tabla = df.iloc[1:, start_col:start_col+2].copy()  
    tabla.columns = ['País', df.iloc[3, start_col+1]]  
    tabla = tabla.dropna(subset=['País'])
    return tabla.reset_index(drop=True)


poblacion = procesar_columna_ajustada(df_mercado_clean, 1).rename(
    columns={df_mercado_clean.iloc[3,2]: 'Población'})

consumo = procesar_columna_ajustada(df_mercado_clean, 4).rename(
    columns={df_mercado_clean.iloc[3,5]: 'Consumo_Tn'})

mercado = procesar_columna_ajustada(df_mercado_clean, 7).rename(
    columns={df_mercado_clean.iloc[3,8]: 'Mercado_USD'})

def limpiar_final(df):
    df['País'] = df['País'].astype(str).str.strip()
    df = df.replace({
        'Reino Unido (Irlanda del Norte)': 'Reino Unido',
        'Chequia': 'República Checa',
        'México': 'México' 
    })
    if 'Mercado_USD' in df.columns:
        df['Mercado_USD'] = df['Mercado_USD'].astype(str).str.replace('[$,]', '', regex=True).replace('nan', np.nan).astype(float)
    return df

df_final = (
    limpiar_final(poblacion)
    .merge(limpiar_final(consumo), on='País', how='outer')
    .merge(limpiar_final(mercado), on='País', how='outer')
    .sort_values('Población', ascending=False)
    .reset_index(drop=True)
)

df_final[['Población', 'Consumo_Tn']] = df_final[['Población', 'Consumo_Tn']].apply(pd.to_numeric, errors='coerce')

In [12]:
df_final.to_csv('../data/mercado.csv', index=False)